In [91]:
import numpy as np
import matplotlib.pyplot
import mnist
import scipy
import scipy.sparse.linalg
import scipy.misc
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from numpy import linalg as LA
import pdb

In [92]:
trainX, trainY = mnist.load_mnist("training", None, './MNIST');
testX, testY = mnist.load_mnist("testing", None, './MNIST');

print("MNIST loaded")

MNIST loaded


In [93]:
class PCA:
    
    def __init__(self):
        self.sigma = 0;
        
    def load_train(self, trainX):
        s = trainX.shape;
        self.d = s[1] * s[2];
        self.N = s[0];
        self.X = np.reshape(trainX, (self.N, self.d));
        print("PCA : Training data loaded...")
        
    def compSigma(self):
        self.Sigma = self.X.transpose().dot(self.X);
        self.w, self.v = LA.eig(self.Sigma);
        print("PCA : Sigma Computed...")
    
    def decomposeInput(self, k):
        weights = np.zeros((self.N, k));
        for i in range(0, k):
            weights[:, i] = np.reshape(self.X.dot(np.reshape(self.v[:,i], (784,1))), self.N);
        print("PCA : Data decomposed to " + str(k) + " dimensions...")
        return weights
    
    def decompose(self, procdata, k):
        weights = np.zeros((procdata.shape[0], k));
        for i in range(0, k):
            weights[:, i] = np.reshape(procdata.dot(np.reshape(self.v[:,i], (784,1))), procdata.shape[0]);
        print("PCA : Input data decomposed to " + str(k) + " dimensions...")
        return weights

In [94]:
testX = np.reshape(testX, (testX.shape[0], model.d-1));
model = PCA();
model.load_train(trainX);
model.compSigma();
pca50Xtr = model.decomposeInput(50);
pca50Xte = model.decompose(testX, 50);

PCA : Training data loaded...
PCA : Sigma Computed...
PCA : Data decomposed to 50 dimensions...
PCA : Input data decomposed to 50 dimensions...


In [198]:
class neuralnet:
    def __init__(self, connect, nNeurons_FC):
        self.connect = connect;
        self.nNeurons_FC = nNeurons_FC;
        self.nLayers = len(connect);
        print("NN : Running Neural Net ", [connect[i] for i in range(self.nLayers)])
    
    def load_train(self, trainX, trainY):
        s = trainX.shape;
        self.d = s[1] + 1;
        self.N = s[0];
        self.K = np.max(trainY)+1
        self.trainX = np.insert(trainX, 0, 1, axis=1);
        
        self.trainY1 = trainY;
        self.trainY = np.zeros((self.N, self.K),dtype=np.float)
        for i in range(0,self.K):
            temp = np.reshape(np.array(trainY == i, dtype=float), (self.N, 1));
            self.trainY[:,i] = np.asmatrix(np.reshape(temp, self.N))
            
        print("NN : Training data loaded...")
    
    def init_net(self):
        self.layers = [np.zeros((self.nNeurons_FC[0], self.d))];
        for i in range(1,self.nLayers):
            if self.connect[i] is "FC":
                self.layers.append(np.zeros((self.nNeurons_FC[i], self.nNeurons_FC[i-1])));
            else:
                self.layers.append(np.zeros(0));
        
        self.layervals = []
        for i in range(self.nLayers):
            if self.connect[i] is "LOSS":
                self.layervals.append(np.zeros(1));
            else:
                self.layervals.append(np.zeros((self.N, self.nNeurons_FC[i])));
        print("NN : Neural Net initialized...")
    
    def forward(self):
        curval = self.trainX;
        for i in range(self.nLayers):
            if self.connect[i] is "FC":
                curval = curval.dot(self.layers[i].transpose());
                self.layervals[i] = curval
            elif self.connect[i] is "TANH":
                curval = np.tanh(curval);
                self.layervals[i] = curval
            elif self.connect[i] is "LOSS":
                curval = (curval - self.trainY)*(curval - self.trainY);
                curval = np.sum(curval, axis = 1);
                self.layervals[i] = curval
                self.loss = np.sum(curval);
    
    def compGradLoss(self):
        
        
        print("NN : forwarded " + str(self.nLayers) + " layers...")

In [199]:
connect = ["FC", "TANH", "FC", "LOSS"];
nNeurons_FC = [500, 500, 10, 1];

NN = neuralnet(connect, nNeurons_FC);

NN.load_train(pca50Xtr, trainY);
NN.init_net();
NN.forward();

('NN : Running Neural Net ', ['FC', 'TANH', 'FC', 'LOSS'])
NN : Training data loaded...
NN : Neural Net initialized...
NN : forwarded 4 layers...


In [140]:
NN.layervals[1].shape

(60000, 10)